In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2021-ai-w4-p2/sample.csv
/kaggle/input/2021-ai-w4-p2/train.csv
/kaggle/input/2021-ai-w4-p2/test.csv


In [2]:
train = pd.read_csv("/kaggle/input/2021-ai-w4-p2/train.csv")
test = pd.read_csv("/kaggle/input/2021-ai-w4-p2/test.csv")
submit = pd.read_csv("/kaggle/input/2021-ai-w4-p2/sample.csv")

train.head(5)

,N,P,K,temperature,humidity,ph,rainfall,label
0,22,36,16,30.581395,50.771481,8.184229,64.585596,mothbeans
1,18,27,41,22.365094,92.308824,7.175344,104.821633,pomegranate
2,61,68,50,35.214628,91.497251,6.793245,243.074507,papaya
3,26,72,22,28.767949,37.577921,4.674942,91.720849,pigeonpeas
4,2,140,197,22.697801,92.822234,5.534567,105.050823,apple


In [3]:
print(train.info())
print()
print(test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1650 entries, 0 to 1649
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   N            1650 non-null   int64  
 1   P            1650 non-null   int64  
 2   K            1650 non-null   int64  
 3   temperature  1650 non-null   float64
 4   humidity     1650 non-null   float64
 5   ph           1650 non-null   float64
 6   rainfall     1650 non-null   float64
 7   label        1650 non-null   object 
dtypes: float64(4), int64(3), object(1)
memory usage: 103.2+ KB
None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 550 entries, 0 to 549
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   N            550 non-null    int64  
 1   P            550 non-null    int64  
 2   K            550 non-null    int64  
 3   temperature  550 non-null    float64
 4   humidity     550 non-null    float64
 5   p

In [4]:
train.describe()

,N,P,K,temperature,humidity,ph,rainfall
count,1650.000000,1650.000000,1650.000000,1650.000000,1650.000000,1650.000000,1650.000000
mean,50.873333,53.345455,48.026667,25.515033,71.665353,6.474123,102.509781
std,37.314088,33.200469,50.954520,5.035143,21.971147,0.777618,55.298782
min,0.000000,5.000000,5.000000,8.825675,14.273280,3.504752,20.360011
25%,20.250000,28.000000,20.000000,22.713912,60.521146,5.956121,63.659862
50%,37.000000,51.000000,31.000000,25.546615,80.519728,6.431830,93.516755
75%,85.000000,68.000000,48.000000,28.435373,89.894716,6.921558,122.340263
max,140.000000,145.000000,205.000000,43.675493,99.981876,9.935091,298.560117


In [5]:
X_train = train.drop('label', axis = 1)
y_train = train['label']
X_test = test

In [6]:
X_train.columns

Index(['N', 'P', 'K', 'temperature', 'humidity', 'ph', 'rainfall'], dtype='object')

In [7]:
from sklearn.preprocessing import MinMaxScaler

# sc_cols = ['N', 'P', 'K','temperature', 'humidity', 'rainfall']
# sc = MinMaxScaler()
# X_train[sc_cols] = sc.fit_transform(X_train[sc_cols])
# X_test[sc_cols] = sc.transform(X_test[sc_cols])

sc = MinMaxScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [8]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train = le.fit_transform(y_train)

In [9]:
import torch
import torch.optim as optim
import torch.nn.functional as F

torch.manual_seed(1)

In [10]:
X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)
y_train = torch.LongTensor(y_train) #label encoding -> array, argmax 때문에 long

### multi class classification

H = softmax(XW+b)<br/>
cost = cross_entropy(H, y)<br/>

y_pred = argmax(softmax(X_test@W+b))

In [11]:
nb_feature = X_train.size()[1]
nb_class = nb_class = y_train.unique().size()[0]
W = torch.zeros((nb_feature, nb_class),requires_grad = True)
b = torch.zeros(1,requires_grad = True)  # b = tensor.zeros((1, nb_class),requires_grad = True)
optimizer = optim.SGD([W, b], lr=0.1)

nb_ephocs = 200000
for ephoc in range(nb_ephocs + 1):
    H = F.softmax(X_train@W+b, dim = 1)
    cost = F.cross_entropy(X_train@W+b, y_train)
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    if ephoc % 1000 == 0 :
        predict = torch.argmax(H, dim = 1)
        correct_pred = predict.float() == y_train
        accuracy = correct_pred.sum().item() / len(correct_pred) * 100
        print("{}: {:.4f} {:.3f}%".format(ephoc, cost.item(), accuracy))


0: 3.0910 4.303%
1000: 2.3123 58.485%
2000: 1.9185 69.818%
3000: 1.6656 73.758%
4000: 1.4876 77.091%
5000: 1.3546 80.182%
6000: 1.2507 82.606%
7000: 1.1670 84.303%
8000: 1.0979 86.242%
9000: 1.0395 87.636%
10000: 0.9895 88.667%
11000: 0.9461 89.515%
12000: 0.9079 89.879%
13000: 0.8740 90.364%
14000: 0.8436 90.545%
15000: 0.8162 90.848%
16000: 0.7913 91.212%
17000: 0.7686 91.455%
18000: 0.7478 91.758%
19000: 0.7286 91.879%
20000: 0.7108 92.242%
21000: 0.6943 92.545%
22000: 0.6789 92.667%
23000: 0.6645 92.848%
24000: 0.6510 93.091%
25000: 0.6383 93.212%
26000: 0.6264 93.394%
27000: 0.6151 93.394%
28000: 0.6044 93.515%
29000: 0.5943 93.515%
30000: 0.5846 93.697%
31000: 0.5755 93.576%
32000: 0.5667 93.576%
33000: 0.5584 93.697%
34000: 0.5504 93.758%
35000: 0.5428 93.758%
36000: 0.5355 93.758%
37000: 0.5285 93.879%
38000: 0.5218 93.879%
39000: 0.5153 94.000%
40000: 0.5091 94.000%
41000: 0.5031 94.121%
42000: 0.4973 94.242%
43000: 0.4918 94.303%
44000: 0.4864 94.303%
45000: 0.4812 94.303%
46

In [12]:
predict = torch.argmax(F.softmax(X_test@W+b, dim = 1), dim = 1)
predict = predict.detach().numpy()
predict = le.inverse_transform(predict)
predict

array(['muskmelon', 'watermelon', 'papaya', 'papaya', 'apple', 'mango',
       'apple', 'blackgram', 'mungbean', 'lentil', 'blackgram', 'coconut',
       'pomegranate', 'jute', 'coconut', 'pomegranate', 'apple', 'cotton',
       'papaya', 'muskmelon', 'coffee', 'papaya', 'orange', 'papaya',
       'chickpea', 'jute', 'mungbean', 'orange', 'pigeonpeas', 'rice',
       'pomegranate', 'mothbeans', 'jute', 'lentil', 'jute', 'blackgram',
       'jute', 'chickpea', 'chickpea', 'kidneybeans', 'papaya', 'mango',
       'blackgram', 'maize', 'mungbean', 'maize', 'pigeonpeas', 'coconut',
       'muskmelon', 'maize', 'mothbeans', 'coffee', 'grapes', 'mungbean',
       'coffee', 'kidneybeans', 'cotton', 'apple', 'banana', 'lentil',
       'watermelon', 'coconut', 'lentil', 'orange', 'mungbean',
       'blackgram', 'orange', 'rice', 'muskmelon', 'pigeonpeas',
       'muskmelon', 'coconut', 'jute', 'banana', 'blackgram', 'papaya',
       'banana', 'cotton', 'watermelon', 'orange', 'coffee', 'chickpe

In [13]:
submit['label'] = predict
submit.to_csv("submit.csv", mode = 'w', index = False)